In [270]:
import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams
import seaborn as sns
import pandas as pd
import numpy as np
import sys
from scipy.stats import randint as sp_randint
import sklearn as sk
from sklearn import metrics, decomposition
from sklearn.model_selection import train_test_split, ShuffleSplit, learning_curve, RandomizedSearchCV, GridSearchCV
from sklearn.preprocessing import StandardScaler, Imputer
from sklearn.feature_selection import SelectKBest
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.calibration import CalibratedClassifierCV
import h2o
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.estimators.random_forest import H2ORandomForestEstimator
from h2o.model.metrics_base import H2OBinomialModelMetrics
import os
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import MultinomialNB
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn import svm

In [2]:
ronaldo = pd.read_csv('data.csv', index_col=0)
print(ronaldo.head())

   match_event_id  location_x  location_y  remaining_min  power_of_shot  \
0            10.0       167.0        72.0           10.0            1.0   
1            12.0      -157.0         0.0           10.0            1.0   
2            35.0      -101.0       135.0            7.0            1.0   
3            43.0       138.0       175.0            6.0            1.0   
4           155.0         0.0         0.0            NaN            2.0   

   knockout_match game_season  remaining_sec  distance_of_shot  is_goal  \
0             0.0     2000-01           27.0              38.0      NaN   
1             0.0     2000-01           22.0              35.0      0.0   
2             0.0     2000-01           45.0              36.0      1.0   
3             0.0     2000-01           52.0              42.0      0.0   
4             0.0     2000-01           19.0              20.0      1.0   

         ...                         lat/lng type_of_shot  \
0        ...          45.539131, -122

In [3]:
sample_sub= pd.read_csv('sample_submission.csv')
sample_sub.head()

,shot_id_number,is_goal
0,1,0.1
1,8,0.1
2,17,0.1
3,20,0.1
4,33,0.1


In [4]:
print(ronaldo.columns)

Index(['match_event_id', 'location_x', 'location_y', 'remaining_min',
       'power_of_shot', 'knockout_match', 'game_season', 'remaining_sec',
       'distance_of_shot', 'is_goal', 'area_of_shot', 'shot_basics',
       'range_of_shot', 'team_name', 'date_of_game', 'home/away',
       'shot_id_number', 'lat/lng', 'type_of_shot', 'type_of_combined_shot',
       'match_id', 'team_id', 'remaining_min.1', 'power_of_shot.1',
       'knockout_match.1', 'remaining_sec.1', 'distance_of_shot.1'],
      dtype='object')


In [5]:
ronaldo['date_of_game']=pd.to_datetime(ronaldo['date_of_game'])
ronaldo['date_today'] = pd.to_datetime('2019-07-20 00:00:00')
ronaldo['num_days'] = (ronaldo['date_today'] - ronaldo['date_of_game']).dt.days

for i in range(len(ronaldo['shot_id_number'])):
    ronaldo['shot_id_number'][i]=i+1
    if(ronaldo['lat/lng'][i]=='42.982923, -71.446094'):
        ronaldo['home/away'][i]=1
    else:
        ronaldo['home/away'][i]=0

C:\Users\rishabh.ranjan\AppData\Local\Continuum\anaconda3\envs\env_full\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\rishabh.ranjan\AppData\Local\Continuum\anaconda3\envs\env_full\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\rishabh.ranjan\AppData\Local\Continuum\anaconda3\envs\env_full\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/st

In [6]:
ronaldo.drop(['match_event_id', 'game_season','team_name', 'date_of_game', 'lat/lng', 'match_id', 'team_id', 'remaining_min.1',
       'power_of_shot.1', 'knockout_match.1', 'remaining_sec.1',
       'distance_of_shot.1','date_today'], axis=1, inplace=True)

In [7]:
print(ronaldo.columns)

Index(['location_x', 'location_y', 'remaining_min', 'power_of_shot',
       'knockout_match', 'remaining_sec', 'distance_of_shot', 'is_goal',
       'area_of_shot', 'shot_basics', 'range_of_shot', 'home/away',
       'shot_id_number', 'type_of_shot', 'type_of_combined_shot', 'num_days'],
      dtype='object')


In [8]:
ronaldo['type_of_shot'] = ronaldo.type_of_shot.fillna('0')
ronaldo['type_of_combined_shot'] = ronaldo.type_of_combined_shot.fillna('0')

In [9]:
columnsWithNa = ['location_x', 'location_y', 'remaining_min', 'power_of_shot', 'remaining_sec', 'distance_of_shot']
for column in columnsWithNa :
    print(ronaldo[column].mean())
    ronaldo[column].fillna(float(ronaldo[column].mean()), inplace = True)

7.383876043234369
91.12693349795933
4.883233224643899
2.51935914552737
28.329381850668316
33.448884311706145


In [10]:
ronaldo['knockout_match'].fillna(ronaldo['knockout_match'].mode()[0], inplace = True)

In [11]:
columnsWithNa = ['area_of_shot', 'shot_basics', 'range_of_shot']
for column in columnsWithNa :
    print(ronaldo[column].mode()[0])
    ronaldo[column].fillna(str(ronaldo[column].mode()[0]), inplace = True)

Center(C)
Mid Range
Less Than 8 ft.


In [12]:
print(ronaldo.head())
ronaldo.to_csv('ronaldo.csv')

   location_x  location_y  remaining_min  power_of_shot  knockout_match  \
0       167.0        72.0      10.000000            1.0             0.0   
1      -157.0         0.0      10.000000            1.0             0.0   
2      -101.0       135.0       7.000000            1.0             0.0   
3       138.0       175.0       6.000000            1.0             0.0   
4         0.0         0.0       4.883233            2.0             0.0   

   remaining_sec  distance_of_shot  is_goal           area_of_shot  \
0           27.0              38.0      NaN          Right Side(R)   
1           22.0              35.0      0.0           Left Side(L)   
2           45.0              36.0      1.0   Left Side Center(LC)   
3           52.0              42.0      0.0  Right Side Center(RC)   
4           19.0              20.0      1.0              Center(C)   

  shot_basics    range_of_shot home/away  shot_id_number type_of_shot  \
0   Mid Range        16-24 ft.         0             1.

In [13]:
ronaldo1=ronaldo

In [14]:
ronaldo1=ronaldo1[ronaldo1.is_goal.isna() == False]
ronaldo_test=ronaldo[ronaldo.is_goal.isna() == True]

In [15]:
ronaldo1.dtypes

location_x               float64
location_y               float64
remaining_min            float64
power_of_shot            float64
knockout_match           float64
remaining_sec            float64
distance_of_shot         float64
is_goal                  float64
area_of_shot              object
shot_basics               object
range_of_shot             object
home/away                 object
shot_id_number           float64
type_of_shot              object
type_of_combined_shot     object
num_days                 float64
dtype: object

In [16]:
print(ronaldo1.head())
print(ronaldo_test.head())

   location_x  location_y  remaining_min  power_of_shot  knockout_match  \
1      -157.0         0.0      10.000000            1.0             0.0   
2      -101.0       135.0       7.000000            1.0             0.0   
3       138.0       175.0       6.000000            1.0             0.0   
4         0.0         0.0       4.883233            2.0             0.0   
5      -145.0       -11.0       9.000000            3.0             0.0   

   remaining_sec  distance_of_shot  is_goal           area_of_shot  \
1           22.0              35.0      0.0           Left Side(L)   
2           45.0              36.0      1.0   Left Side Center(LC)   
3           52.0              42.0      0.0  Right Side Center(RC)   
4           19.0              20.0      1.0              Center(C)   
5           32.0              34.0      0.0           Left Side(L)   

  shot_basics    range_of_shot home/away  shot_id_number type_of_shot  \
1   Mid Range         8-16 ft.         0             2.

In [17]:
ronaldo1 = pd.get_dummies(ronaldo1, dummy_na=False)
ronaldo_test = pd.get_dummies(ronaldo_test, dummy_na=False)

In [18]:
shot_id=ronaldo_test.shot_id_number.tolist()
shot_id_numb=sample_sub.shot_id_number.tolist()
l1=[]
for s in shot_id:
    if(s not in shot_id_numb):
        l1.append(int(s-1))
l2=[]
for i in range(len(ronaldo_test)):
    if(ronaldo_test.shot_id_number.index[i] not in l1):
        print(ronaldo_test.shot_id_number.index[i])
        l2.append(i)

0
7
16
19
32
33
34
35
36
37
44
49
54
59
65
66
70
79
84
85
94
103
112
122
125
132
140
143
149
151
152
155
158
164
171
181
191
192
195
201
209
210
227
256
259
263
266
278
281
292
298
299
300
316
326
327
334
345
347
364
389
390
391
396
417
427
433
438
439
455
458
471
479
490
496
508
510
522
532
543
545
554
557
559
572
590
595
596
600
604
616
627
633
645
648
650
653
655
656
668
669
692
693
698
710
717
721
722
724
733
734
736
745
746
765
766
770
772
773
782
783
788
791
803
808
811
812
816
819
824
825
828
854
858
861
869
875
879
885
888
890
896
901
902
907
908
919
921
923
931
933
936
940
942
949
951
953
962
974
978
980
982
984
988
994
997
1007
1018
1022
1025
1028
1033
1035
1037
1038
1050
1051
1065
1086
1091
1093
1097
1098
1101
1117
1130
1132
1134
1137
1138
1141
1150
1151
1154
1164
1166
1171
1173
1179
1199
1201
1226
1227
1228
1236
1240
1255
1278
1292
1294
1318
1319
1320
1321
1333
1350
1372
1374
1379
1381
1384
1398
1402
1427
1433
1443
1447
1465
1475
1477
1484
1490
1493
1499
1500
1502
1518
1523

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)




13785
13786
13791
13794
13797
13806
13808
13810
13814
13816
13817
13821
13823
13828
13831
13839
13840
13867
13870
13879
13887
13889
13890
13894
13906
13909
13911
13912
13922
13928
13935
13945
13949
13950
13953
13957
13968
13979
13980
13985
13986
13988
13991
13993
14000
14001
14003
14005
14011
14020
14023
14044
14046
14056
14058
14059
14067
14069
14073
14075
14088
14114
14115
14127
14130
14133
14136
14137
14138
14151
14152
14154
14155
14163
14164
14170
14173
14180
14185
14191
14200
14209
14213
14230
14235
14240
14244
14248
14250
14261
14264
14266
14269
14275
14278
14280
14288
14289
14292
14294
14296
14297
14309
14311
14314
14317
14318
14324
14330
14333
14340
14342
14343
14345
14346
14348
14351
14357
14360
14362
14373
14375
14380
14397
14409
14411
14414
14422
14424
14435
14443
14454
14464
14468
14470
14474
14478
14479
14487
14490
14500
14519
14521
14522
14536
14537
14549
14560
14574
14585
14591
14600
14601
14603
14605
14626
14640
14651
14661
14664
14672
14674
14678
14682
14685
14704
147

22370
22372
22390
22403
22408
22410
22427
22435
22443
22444
22450
22462
22463
22466
22471
22484
22496
22503
22507
22515
22521
22526
22542
22548
22550
22556
22561
22566
22571
22574
22577
22578
22579
22580
22590
22592
22596
22599
22601
22602
22615
22616
22621
22622
22623
22627
22639
22640
22652
22659
22671
22673
22688
22697
22704
22708
22710
22713
22714
22728
22729
22732
22749
22753
22755
22757
22766
22767
22768
22775
22777
22779
22788
22796
22798
22813
22817
22820
22822
22840
22846
22847
22861
22874
22876
22886
22891
22898
22900
22906
22908
22925
22926
22929
22933
22936
22938
22948
22950
22954
22964
22965
22967
22971
22974
22980
22987
22993
22995
23010
23027
23030
23034
23037
23041
23048
23050
23052
23056
23057
23062
23063
23066
23070
23074
23076
23089
23093
23096
23102
23106
23116
23121
23123
23125

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



27436
27440
27443
27444
27447
27453
27458
27464
27477
27478
27480
27482
27485
27497
27504
27506
27508
27513
27526
27534
27543
27544
27555
27562
27568
27584
27588
27604
27607
27608
27610
27620
27625
27628
27644
27650
27653
27655
27662
27672
27676
27677
27679
27680
27683
27687
27705
27708
27719
27721
27732
27733
27738
27745
27750
27754
27776
27782
27794
27804
27810
27814
27822
27855
27872
27882
27884
27886
27898
27920
27923
27926
27928
27931
27942
27944
27951
27956
27959
27965
27967
27979
27983
27993
27994
28001
28003
28005
28027
28041
28042
28054
28060
28062
28076
28089
28092
28094
28095
28097
28098
28110
28114
28122
28124
28125
28128
28130
28138
28145
28152
28154
28159
28165
28166
28175
28184
28186
28194
28202
28205
28208
28209
28210
28215
28233
28234
28247
28254
28265
28276
28285
28287
28288
28289
28295
28299
28305
28306
28308
28326
28335
28344
28354
28360
28361
28373
28379
28380
28392
28393
28404
28412
28425
28427
28428
28441
28444
28456
28458
28461
28477
28489
28491
28498
28499
2850

In [19]:
ronaldo_sub = ronaldo_test.iloc[l2]
ronaldo_sub.head()

,location_x,location_y,remaining_min,power_of_shot,knockout_match,remaining_sec,distance_of_shot,is_goal,shot_id_number,num_days,...,type_of_shot_shot - 7,type_of_shot_shot - 8,type_of_shot_shot - 9,type_of_combined_shot_0,type_of_combined_shot_shot - 0,type_of_combined_shot_shot - 1,type_of_combined_shot_shot - 2,type_of_combined_shot_shot - 3,type_of_combined_shot_shot - 4,type_of_combined_shot_shot - 5
0,167.0,72.0,10.0,1.0,0.0,27.0,38.0,NaN,1.0,6836.0,...,0,0,0,1,0,0,0,0,0,0
7,1.0,28.0,8.0,3.0,0.0,5.0,22.0,NaN,8.0,6836.0,...,0,0,0,0,0,0,0,1,0,0
16,0.0,0.0,0.0,1.0,0.0,1.0,20.0,NaN,17.0,6835.0,...,0,0,0,0,0,0,0,0,1,0
19,0.0,0.0,10.0,3.0,0.0,46.0,20.0,NaN,20.0,6835.0,...,0,0,0,1,0,0,0,0,0,0
32,163.0,76.0,11.0,1.0,0.0,26.0,37.0,NaN,33.0,6832.0,...,0,0,0,1,0,0,0,0,0,0


In [20]:
print(ronaldo1.head())
ronaldo1.to_csv('ronaldo1.csv')
ronaldo_test.to_csv('ronaldo_test.csv')

   location_x  location_y  remaining_min  power_of_shot  knockout_match  \
1      -157.0         0.0      10.000000            1.0             0.0   
2      -101.0       135.0       7.000000            1.0             0.0   
3       138.0       175.0       6.000000            1.0             0.0   
4         0.0         0.0       4.883233            2.0             0.0   
5      -145.0       -11.0       9.000000            3.0             0.0   

   remaining_sec  distance_of_shot  is_goal  shot_id_number  num_days  \
1           22.0              35.0      0.0             2.0    6836.0   
2           45.0              36.0      1.0             3.0    6836.0   
3           52.0              42.0      0.0             4.0    6836.0   
4           19.0              20.0      1.0             5.0    6836.0   
5           32.0              34.0      0.0             6.0    6836.0   

                ...                type_of_shot_shot - 7  \
1               ...                               

In [21]:
Y=ronaldo1['is_goal']
X=ronaldo1[ronaldo1.columns.difference(['is_goal','shot_id_number'])]

In [22]:
X = X.as_matrix()
y = Y.tolist()

C:\Users\rishabh.ranjan\AppData\Local\Continuum\anaconda3\envs\env_full\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [25]:
imputer = Imputer(strategy='median')
pca = decomposition.PCA()
rf = RandomForestClassifier(class_weight='balanced', 
                            n_jobs=-1, 
                            n_estimators=300)

steps = [('imputer', imputer),
        ('pca', pca),
        ('random_forest', rf)]

pipeline = Pipeline(steps)
parameters = dict(
                pca__n_components=[10, 15, 20, 40, 70, 93], 
                random_forest__max_features=['auto', 'log2']
                )

gs = GridSearchCV(pipeline, param_grid=parameters)
gs.fit(X_train, y_train)

C:\Users\rishabh.ranjan\AppData\Local\Continuum\anaconda3\envs\env_full\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\rishabh.ranjan\AppData\Local\Continuum\anaconda3\envs\env_full\lib\site-packages\sklearn\model_selection\_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
C:\Users\rishabh.ranjan\AppData\Local\Continuum\anaconda3\envs\env_full\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarni

C:\Users\rishabh.ranjan\AppData\Local\Continuum\anaconda3\envs\env_full\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\rishabh.ranjan\AppData\Local\Continuum\anaconda3\envs\env_full\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\rishabh.ranjan\AppData\Local\Continuum\anaconda3\envs\env_full\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=

GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('imputer', Imputer(axis=0, copy=True, missing_values='NaN', strategy='median', verbose=0)), ('pca', PCA(copy=True, iterated_power='auto', n_components=None, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('random_forest', RandomForestClassifier(bootstrap=True, class_weight=...ators=300, n_jobs=-1, oob_score=False,
            random_state=None, verbose=0, warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'pca__n_components': [10, 15, 20, 40, 70, 93], 'random_forest__max_features': ['auto', 'log2']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [26]:
print(gs.best_score_)
print(gs.best_params_)

0.6166081871345029
{'pca__n_components': 40, 'random_forest__max_features': 'log2'}


In [156]:
rf = gs.best_estimator_
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.62      0.83      0.71      3982
         1.0       0.66      0.38      0.48      3347

   micro avg       0.63      0.63      0.63      7329
   macro avg       0.64      0.61      0.59      7329
weighted avg       0.63      0.63      0.60      7329

0.626552053486151


In [28]:
y_pred = rf.predict_proba(ronaldo_sub[ronaldo_sub.columns.difference(['is_goal','shot_id_number'])])[:, 1]

In [29]:
sub_pred=y_pred.tolist()
sub=sample_sub
for i in range(len(sub)):
    sub['is_goal'][i]=sub_pred[i]

C:\Users\rishabh.ranjan\AppData\Local\Continuum\anaconda3\envs\env_full\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [30]:
sub.to_csv('sub1.csv',index=False)

In [167]:
models = [
    RandomForestClassifier(class_weight='balanced', n_jobs=-1, n_estimators=300),
    LinearSVC(),
    LogisticRegression(random_state=0),
    XGBClassifier(class_weight='balanced', n_jobs=-1, n_estimators=300)
]
for model in models:
    imputer = Imputer(strategy='median')
    pca = decomposition.PCA()
    param = {'max_depth':2, 'eta':1, 'objective':'binary:logistic' }
    num_round = 1
    mod = model

    steps = [('imputer', imputer),
            ('pca', pca),
            ('mod', mod)]

    pipeline = Pipeline(steps)
    parameters = dict(
                    pca__n_components=[30,40,42,93], 
                    )

    gs = GridSearchCV(pipeline, param_grid=parameters)
    gs.fit(X_train, y_train)
    print(model)
    print(gs.best_score_)
    print(gs.best_params_)

C:\Users\rishabh.ranjan\AppData\Local\Continuum\anaconda3\envs\env_full\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\rishabh.ranjan\AppData\Local\Continuum\anaconda3\envs\env_full\lib\site-packages\sklearn\model_selection\_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
C:\Users\rishabh.ranjan\AppData\Local\Continuum\anaconda3\envs\env_full\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarni

RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=300, n_jobs=-1, oob_score=False,
            random_state=None, verbose=0, warm_start=False)
0.6187134502923977
{'pca__n_components': 42}


C:\Users\rishabh.ranjan\AppData\Local\Continuum\anaconda3\envs\env_full\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\rishabh.ranjan\AppData\Local\Continuum\anaconda3\envs\env_full\lib\site-packages\sklearn\model_selection\_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
C:\Users\rishabh.ranjan\AppData\Local\Continuum\anaconda3\envs\env_full\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarni

C:\Users\rishabh.ranjan\AppData\Local\Continuum\anaconda3\envs\env_full\lib\site-packages\sklearn\svm\base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)
0.5374269005847954
{'pca__n_components': 40}


C:\Users\rishabh.ranjan\AppData\Local\Continuum\anaconda3\envs\env_full\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\rishabh.ranjan\AppData\Local\Continuum\anaconda3\envs\env_full\lib\site-packages\sklearn\model_selection\_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
C:\Users\rishabh.ranjan\AppData\Local\Continuum\anaconda3\envs\env_full\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarni

C:\Users\rishabh.ranjan\AppData\Local\Continuum\anaconda3\envs\env_full\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=0, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)
0.6308187134502924
{'pca__n_components': 30}


C:\Users\rishabh.ranjan\AppData\Local\Continuum\anaconda3\envs\env_full\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\rishabh.ranjan\AppData\Local\Continuum\anaconda3\envs\env_full\lib\site-packages\sklearn\model_selection\_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
C:\Users\rishabh.ranjan\AppData\Local\Continuum\anaconda3\envs\env_full\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarni

XGBClassifier(base_score=0.5, booster='gbtree', class_weight='balanced',
       colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=300, n_jobs=-1,
       nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=None, subsample=1, verbosity=1)
0.6352046783625731
{'pca__n_components': 30}


In [ ]:
imputer = Imputer(strategy='median')
pca = decomposition.PCA()
num_round = 5
xgb = GradientBoostingClassifier( learning_rate=0.0001, n_estimators=500,max_depth=60, min_samples_split=1200, min_samples_leaf=60, subsample=0.85, random_state=10, max_features=29,
warm_start=True)
steps = [('imputer', imputer),('pca', pca),('xgb', xgb)]

pipeline = Pipeline(steps)
parameters = dict(pca__n_components=range(30,44),)

gs = GridSearchCV(pipeline, param_grid=parameters)
gs.fit(X_train, y_train)
print(gs.best_score_)
print(gs.best_params_)

C:\Users\rishabh.ranjan\AppData\Local\Continuum\anaconda3\envs\env_full\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\rishabh.ranjan\AppData\Local\Continuum\anaconda3\envs\env_full\lib\site-packages\sklearn\model_selection\_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
C:\Users\rishabh.ranjan\AppData\Local\Continuum\anaconda3\envs\env_full\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarni

In [272]:
xgb=gs.best_estimator_
xgb.fit(X_train, y_train)

y_pred = xgb.predict(X_test)
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

C:\Users\rishabh.ranjan\AppData\Local\Continuum\anaconda3\envs\env_full\lib\site-packages\sklearn\model_selection\_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
C:\Users\rishabh.ranjan\AppData\Local\Continuum\anaconda3\envs\env_full\lib\site-packages\sklearn\svm\base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\rishabh.ranjan\AppData\Local\Continuum\anaconda3\envs\env_full\lib\site-packages\sklearn\svm\base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\rishabh.ranjan\AppData\Local\Continuum\anaconda3\envs\env_full\lib\site-packages\sklearn\svm\base.py:922: ConvergenceWarning: Liblinear failed to converge, increase 

              precision    recall  f1-score   support

         0.0       0.55      0.99      0.71      3982
         1.0       0.71      0.03      0.05      3347

   micro avg       0.55      0.55      0.55      7329
   macro avg       0.63      0.51      0.38      7329
weighted avg       0.62      0.55      0.41      7329

0.5506890435257198


In [273]:
y_pred = xgb.predict_proba(ronaldo_test[ronaldo_test.columns.difference(['is_goal','shot_id_number'])])[:, 1]
sub_pred=y_pred.tolist()
lsub=[]
for i in range(len(sub_pred)):
    lsub.append([ronaldo_test.shot_id_number.index[i],sub_pred[i]])
sub = pd.DataFrame(lsub, columns = ['shot_id_number','is_goal'])
# sub=sample_sub
# for i in range(len(sub)):
#     sub['is_goal'][i]=sub_pred[i]

In [274]:
sub.to_csv('Rishabh_Ranjan_210719_prediction_4.csv',index=False)

In [143]:
lf=gs.best_estimator_.named_steps["xgb"].feature_importances_
lf=lf.tolist()
len(lf)
len(ronaldo_sub[ronaldo_sub.columns.difference(['is_goal','shot_id_number'])].columns)
lff=[]
for i in range(len(lf)):
    lff.append([lf[i],ronaldo_sub[ronaldo_sub.columns.difference(['is_goal','shot_id_number'])].columns[i]])
    lff.sort(reverse = True)
lff

[[0.0997115820646286, 'remaining_min'],
 [0.04368225857615471, 'shot_basics_Goal Area'],
 [0.034205589443445206, 'area_of_shot_Right Side Center(RC)'],
 [0.026802701875567436, 'type_of_shot_shot - 46'],
 [0.026597734540700912, 'range_of_shot_24+ ft.'],
 [0.022020865231752396, 'shot_basics_Penalty Spot'],
 [0.020176492631435394, 'area_of_shot_Left Side(L)'],
 [0.016040761023759842, 'type_of_combined_shot_shot - 0'],
 [0.015734190121293068, 'type_of_shot_shot - 17'],
 [0.014894690364599228, 'type_of_shot_shot - 51'],
 [0.014619909226894379, 'type_of_combined_shot_shot - 2'],
 [0.014212450012564659, 'type_of_shot_shot - 32'],
 [0.013938747346401215, 'type_of_combined_shot_shot - 3'],
 [0.013595045544207096, 'type_of_shot_shot - 19'],
 [0.013586536981165409, 'type_of_shot_shot - 15'],
 [0.013005315326154232, 'type_of_shot_shot - 39'],
 [0.012938059866428375, 'location_y'],
 [0.012707407586276531, 'type_of_shot_shot - 0'],
 [0.011714119464159012, 'type_of_shot_shot - 4'],
 [0.01119055598974